In [3]:
!pip install openai==0.28
!pip install galai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [4]:
import openai
import os
import sqlite3

In [5]:
db_location = '/content/gdrive/MyDrive/Database_results'
db_path = os.path.join(db_location, 'summarization_results.db')
print(db_path)

/content/gdrive/MyDrive/Database_results/summarization_results.db


In [6]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [9]:
#Loading the secrets that include (openAI key, and as well as Hugging Face token)
import os
with open("env.txt", "r") as file:
    for row in file:
        key, value = row.strip().split('=', 1)
        os.environ[key] = value

In [41]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [42]:
cursor.execute('''TRUNCATE table LLM_Answers_OpenAI_Final''')

OperationalError: near "TRUNCATE": syntax error

**GALACTICA**

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS LLM_answers_Prompt1_Galactica_V2
(id INTEGER PRIMARY KEY AUTOINCREMENT,
 claim_number INTEGER,
claim_name TEXT,
compound_name TEXT,
overall_summary TEXT,
question1_galactica TEXT,
answer1_galactica TEXT,
question2_galactica TEXT,
answer2_galactica TEXT,
question3_galactica TEXT,
answer3_galactica TEXT)''')

In [ ]:
import galai as gal
from galai.notebook_utils import *
model = gal.load_model("base")

In [ ]:
#LLM_answers_Prompt1_Galactica_V2
cursor.execute('SELECT claim_number, claim, compound_name, combined_summary, overall_summary, Health_Relationship FROM langchain_results_new')
entries = cursor.fetchall()

def ask_galactica(summary, question):
    #answer_gal1 = model.generate(f"{str(overall_summary)}.{str(question)}")
    answer_galactica = model.generate(f"Question: Please read the data below and answer the question: {str(summary)}.{str(question)}\n\nAnswer:")
    return answer_galactica

for entry in entries:
    claim_number, claim, compound_name, combined_summary, overall_summary, Health_Relationship = entry[:6]


    question1_galactica = f'With the data given above can you confirm if the claim: "{claim}" as true and valid?'
    question2_galactica = f'Given the data above can you provide the nutritional relationship and health effects of \"{compound_name}\" on human health? '
    question3_galactica = f"Given the data above can you confirm if the Health Relationship \"{Health_Relationship}\" between the \"{compound_name}\" and human health is True? "

    answer1_galactica = ask_galactica(overall_summary, question1_galactica)
    answer2_galactica = ask_galactica(overall_summary, question2_galactica)
    answer3_galactica = ask_galactica(overall_summary, question3_galactica)


    cursor.execute('''
    INSERT INTO LLM_Answers_Prompt1_Galactica_V2(claim_number, claim_name, compound_name, overall_summary, health_relationship,question1, answer1_openai, question2, answer2_openai, question1_gal,answer1_gal, question2_gal, answer2_gal
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (claim_number, claim, compound_name, overall_summary, Health_Relationship,question1, answer1_openai, question2, answer2_openai,question1_gal, answer1_gal, question2_gal, answer2_gal))

conn.commit()
conn.close()

**OPENAI**

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS LLM_answers_Prompt1
          (id INTEGER PRIMARY KEY AUTOINCREMENT,
          claim_number INTEGER,
          claim_name TEXT,
          health_relationship TEXT,
          compound_name TEXT,
          overall_summary TEXT,
          question1 TEXT,
          answer1_openai TEXT,
          question2 TEXT,
          answer2_openai TEXT,
          question1_gal TEXT,
          answer1_gal TEXT,
          question2_gal TEXT,
          answer2_gal TEXT)''')

In [ ]:
#LLM_answers_Prompt1
#only answers answer1_openai considered here
cursor.execute('SELECT claim_number, claim, compound_name, combined_summary, overall_summary, Health_Relationship FROM langchain_results_new')
entries = cursor.fetchall()


def ask_openai(prompt):
    #
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=100
    )
    return response['choices'][0]['text'].strip()

def ask_galactica(summary, question):
    #answer_gal1 = model.generate(f"{str(overall_summary)}.{str(question)}")
    answer_gal = model.generate(f"Question: Please read the data below and answer the question: {str(summary)}.{str(question)}\n\nAnswer:")
    return answer_gal

for entry in entries:
    claim_number, claim, compound_name, combined_summary, overall_summary, Health_Relationship = entry[:6]


    question1 = f'Given the overall summary "{overall_summary}", can you confirm if the claim "{claim}" is true?'
    question2 = f"Given the overall summary \"{overall_summary}\", can you confirm if the Health Relationship \"{Health_Relationship}\" between the \"{compound_name}\" and human health is True?"
    question1_gal = f'With the data given above can you confirm if the claim: "{claim}" as true or untrue?'
    question2_gal = f"Given the data above can you confirm if the Health Relationship \"{Health_Relationship}\" between the \"{compound_name}\" and human health is True? "


    answer1_openai = ask_openai(question1)
    answer2_openai = ask_openai(question2)


    answer1_gal = ask_galactica(overall_summary, question1_gal)
    answer2_gal = ask_galactica(overall_summary, question2_gal)


    cursor.execute('''
    INSERT INTO LLM_answers_Prompt1(claim_number, claim_name, compound_name, overall_summary, health_relationship,question1, answer1_openai, question2, answer2_openai, question1_gal,answer1_gal, question2_gal, answer2_gal)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', (claim_number, claim, compound_name, overall_summary, Health_Relationship,
          question1, answer1_openai, question2, answer2_openai,question1_gal, answer1_gal, question2_gal, answer2_gal))

conn.commit()
conn.close()


In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS LLM_Answers_OpenAI_V2
          (id INTEGER PRIMARY KEY AUTOINCREMENT,
          claim_number INTEGER,
          claim_name TEXT,
          health_relationship TEXT,
          compound_name TEXT,
          overall_summary TEXT,
          question1_openai TEXT,
          answer1_openai TEXT,
          question2_openai TEXT,
          answer2_openai TEXT,
          question3_openai TEXT,
          answer3_openai TEXT)''')

In [ ]:
#LLM_Answers_OpenAI_V2
cursor.execute('SELECT claim_number, claim, compound_name, combined_summary, overall_summary, Health_Relationship FROM langchain_results_new')
rows = cursor.fetchall()


def ask_openai(prompt):
    #
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=500
        #answers are huge hence changing token size over here, gotta change to 1000 if colab allows
    )
    return response['choices'][0]['text'].strip()

for entry in entries:
    claim_number, claim, compound_name, combined_summary, overall_summary, Health_Relationship = entry[:6]


    question1_openai = f'Given the overall summary "{overall_summary}", can you confirm if the claim "{claim}" is true? Answer as either Confirmed or Cannot Confirm'
    question2_openai = f"Given the overall summary \"{overall_summary}\", can you provide the relationship between \"{compound_name}\" and its nutritional effect on the human health?"
    question3_openai = f"Given the overall summary \"{overall_summary}\", can you confirm if the Health Relationship \"{Health_Relationship}\" between the \"{compound_name}\" and human health is True?"



    answer1_openai = ask_openai(question1_openai)
    answer2_openai = ask_openai(question2_openai)
    answer3_openai = ask_openai(question3_openai)

    cursor.execute('''
    INSERT INTO LLM_Answers_OpenAI_V2(
        claim_number, claim_name, compound_name, overall_summary, health_relationship,
question1_openai, answer1_openai, question2_openai, answer2_openai, question3_openai,
answer3_openai) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (claim_number, claim, compound_name, overall_summary, Health_Relationship,
question1_openai, answer1_openai, question2_openai, answer2_openai,
question3_openai, answer3_openai))

conn.commit()
conn.close()


In [34]:
cursor.execute('''CREATE TABLE IF NOT EXISTS LLM_Answers_OpenAI_Final
          (id INTEGER PRIMARY KEY AUTOINCREMENT,
          claim_number INTEGER,
          claim_name TEXT,
          health_relationship TEXT,
          compound_name TEXT,
          overall_summary TEXT,
          question1_openai TEXT,
          answer1_openai TEXT,
          question2_openai TEXT,
          answer2_openai TEXT,
          question3_openai TEXT,
          answer3_openai TEXT,
          question4_openai TEXT,
          answer4_openai TEXT)''')

In [35]:
join_query = """ select a1.claim_number, a1.claim_name, a1.health_relationship, a1.compound_name, a1.overall_summary, a2.question1 as question1_openai, a2.answer1_openai as answer1_openai,
a1.question1_openai as question2_openai, a1.answer1_openai as answer2_openai, a1.question2_openai as question3_openai, a1.answer2_openai as answer3_openai, a1.question3_openai as question4_openai,
a1.answer3_openai as answer4_openai
from LLM_Answers_OpenAI_V2 a1
inner join LLM_answers_Prompt1 a2
ON
a1.claim_number = a2.claim_number
"""

In [36]:
cursor.execute(join_query)
entries = cursor.fetchall()

In [27]:
print(entries)

[(1, 'ALA contributes to the maintenance of normal blood cholesterol levels', 'maintenance of normal blood cholesterol concentrations', 'Alpha-linolenic acid (ALA)', ' The text discusses various studies and articles related to the effects of different types of fatty acids on cardiovascular disease risk factors such as blood pressure, cholesterol, and triglycerides. The studies found that diets high in omega-3 fatty acids, particularly EPA and DHA, may have beneficial effects on cardiovascular health. However, more research is needed to fully understand the effects of different types of fatty acids on cardiovascular risk. The text also mentions the importance of a balanced intake of omega-6 and omega-3 fatty acids for optimal health. Additionally, there is conflicting evidence on the potential risks of high intake of alpha-linolenic acid, an essential fatty acid. The text also discusses the effects of dietary factors on blood pressure and the risk of coronary heart disease, as well as t

In [39]:
insert_query = """INSERT INTO LLM_Answers_OpenAI_Final (claim_number, claim_name, health_relationship, compound_name, overall_summary, question1_openai, answer1_openai, question2_openai, answer2_openai, question3_openai, answer3_openai, question4_openai, answer4_openai)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""

In [37]:
try:
    cursor.executemany(insert_query, entries)
    conn.commit()  # Make sure to commit the transaction
except sqlite3.Error as e:
    print("SQLite error:", e)


In [29]:

#cursor.executemany(insert_query, entries)


In [38]:
conn.close()

In [ ]:
#prompt 2

In [ ]:
# for row in rows:
#  claim_number, claim_name, compound_name, combined_summary, overall_summary = row[:5]
#     print(row[4])

In [ ]:
# for row in rows:
#     claim_number, claim_name, compound_name, combined_summary, overall_summary = row[:5]
#     print(row[4])

#     question1 = f"With the given data can you confirm if '{claim_name}' regarding '{compound_name}' accurate?"
#     question2 = f"What are the effects of '{compound_name}' on health?"

#     embeddings = OpenAIEmbeddings()
#     docsearch_overall = FAISS.from_texts(row[4], embeddings)

#     docs_overall = docsearch_overall.similarity_search(question1)
#     chain = load_qa_chain(OpenAI(), chain_type="stuff")
#     answer1_openai = chain.run(input_documents=docs_overall, question=question1)

#     print(question1)
#     print("\nAnswer_OpenAI",answer1_openai)


#     query_2 = f"with the given data,can you provide the relationship between the compound {compound_name} and its nutritional effects on the human body?"
#     print(query_2)
#     #text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200, length_function=len)
#     embeddings = OpenAIEmbeddings()
#     #texts_overall = text_splitter.split_text(str(combined_summary))
#     docsearch_overall = FAISS.from_texts(row[4], embeddings)
#     docs_overall_1 = docsearch_overall.similarity_search(question2)
#     chain = load_qa_chain(OpenAI(), chain_type="stuff")
#     answer2_openai = chain.run(input_documents=docsearch_overall, question=question2)

#     print(question2)
#     print("\nAnswer2_OpenAI",answer2_openai)

#     #Galatica Model Using asking query using the overall summary
#     #model.generate("Question: What is the notch signaling pathway?\n\nAnswer:")
# # 'Question: What is the notch signaling pathway?\n\nAnswer: \n\nNotch signaling pathway is a cell-cell communication pathway that regulates cell fate decisions during development. It is involved in cell proliferation, differentiation, apoptosis, and cell migration. The Notch signaling pathway is activated by the binding of'
#     #answer_gal1 = model.generate(f"Summary:{str(combined_summary)}. Question: {str(question1)}")
#     answer_gal1 = model.generate(f"Question: Please read the data below: {str(overall_summary)}.{str(question1)}")
#     print("\nanswer_gal1",answer_gal1)

#     #answer_gal2=model.generate(f"Summary:{str(combined_summary)}. Question: {str(question2)}")
#     answer_gal2 = model.generate(f"Question: Please read the data below: {str(overall_summary)}.{str(question2)}")
#     print("\nanswer_gal2",answer_gal2)


#     # Insert into LLM_answers table
#     cursor.execute('''
#     INSERT INTO LLM_answers (claim_number, claim_name, compound_name, overall_summary, question1, answer1_openai, question2, answer2_openai, answer1_gal, answer2_gal)
#     VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
#     ''', (str(claim_number), str(claim_name), str(compound_name), str(overall_summary), str(question1), str(answer1_openai), str(question2), str(answer2_openai), str(answer_gal1), str(answer_gal2)))
#     conn.commit()

conn.close()